Check that we can load NN models from this directory

In [ ]:
import sys
sys.path.append('../Climate-Learning')
import general_purpose.utilities as ut


sys.path.append('../Climate-Learning/PLASIM')
import Learn2_new as ln

# log to stdout
import logging
logging.getLogger().level = logging.INFO
logging.getLogger().handlers = [logging.StreamHandler(sys.stdout)]


In [ ]:
def get_arg(run, key, config_dict):
    return run['args'].get(key, ut.extract_nested(config_dict, key))

def get_completed(folder):
    return {k:v for k,v in ut.json2dict(f'{folder}/runs.json').items() if v['status'] == 'COMPLETED'}

def get_checkpoints(run, folder, monitor=None):
    config_dict = ut.json2dict(f'{folder}/config.json')
    nfolds = get_arg(run, 'nfolds', config_dict)
    opt_ckp, fold_subfolder = ln.optimal_checkpoint(f"{folder}/{run['name']}", nfolds, collective=False, metric=monitor or ut.extract_nested(config_dict,'metric'))
    return opt_ckp

def get_model(run, fold, folder, monitor=None):
    opt_ckp = get_checkpoints(run, folder, monitor=monitor)
    load_from = f"{folder}/{run['name']}/fold_{fold}/cp-{opt_ckp[fold]:04d}.ckpt"
    print(f'Loading from {load_from}')
    model = ln.load_model(load_from)
    return model

In [ ]:
folder = 'CNN/r800y'

runs = get_completed(folder)
run = runs['0']

model = get_model(run, 0, folder)
model.summary()